# Profilování a benchmarking
Na začátek je dobré projít, čeho vlastně chceme dosáhnout.

Jednoduše řečeno: **rychlosti**.

**Profilování** nám umožní rozebrat kód a zjistit, kde jsou jeho slabiny (místa, která zabírají nejvíce času). Nasměruje nás tedy k místům kde má smysl uvažovat o optimalizaci a zrychlení.

**Benchmarking** znamená testování (například většího množství různých implementací) na sérii různých vstupů (lišících se velikostí, složitostí, ...). Cílem je zjistit, která implementace je nejrychlejší a pro jaké vstupy.


Při implementaci algoritmů máte poměrně často možnost využít různých postupů, jak něco naimplementovat. A často jsou všechny užitečné, neboť pro různá data a velikosti úloh se hodí různé přístupy. Toto pro nás rozhodne benchmarking.

## Obecně k profilování

Pro Python existuje obrovské množství profilovacích nástrojů. Mnoho z nich je stále také v aktivním vývoji. To proto, že je mnoho faktroů, které je třeba profilovat. Mezi ně patří:
- rychlost (časová náročnost),
- množství použité paměti,
- paměťová propustnost = množství kopírovaných dat,
- GPU profilování.

Profilery navíc přinášejí zpomalení samotného běhu kódu a tím i automatické zkreslení výsledků. Toto zpomalení může být od 1.x krát přes 2x až 5x po 100x v některých případech. Takže jsou i různé varianty profilerů které se liší v kvalitě sezbíraných dat a zpomalením běhu.

## Ukázkový kód pro test profilování

In [1]:
def heavy_calc(X):
    Y = X.copy()
    for i in range(10):
        Y = Y**i
    return Y


def heavy_loop(inputs):
    res = []
    for X in inputs:
        res.append(heavy_calc(X))
    return res


def code_setup():
    from numpy.random import rand

    N = 20
    M = 1000
    print("Will generate {} random arrays".format(N))
    inputs = [rand(M, M) for n in range(N)]
    print("Will calculate now")
    result = heavy_loop(inputs)
    print("Finished calculation")

## Základní profilování: `cProfile`
Jedná se o základní vestavěný profilovací nástroj (knihovna `cProfile`), který je součástí standardní výbavy. Počítá pouze jednotlivá **volání** funkcí a jejich trvání. Není schopen určit, jak dlouho trval který řádek kódu.

- V nejjednodušší verzi stačí použít `cProfile.run()`, jako argument předat kód, který chceme profilovat (např. volání funkce), a soubor, do kterého chceme výsledky uložit (zvykem je použít příponu `.prof`).

In [2]:
import cProfile

cProfile.run("code_setup()", "vysledky_cProfile_code_setup.prof")

Will generate 20 random arrays
Will calculate now
Finished calculation


Výsledky lze vizuálně znázornit pomocí nástroje `snakeviz` (`pip install snakeviz` pokud nemáte). Ten vytvoří webovou stránku s interaktivním grafem a tabulkou.

In [3]:
!snakeviz vysledky_cProfile_code_setup.prof

/bin/bash: line 1: snakeviz: command not found


Výsledky lze také procházet jako textovou tabulku pomocí vestavěného modulu `pstats`.

- `p = pstats.Stats("filename.prof")` vytvoří objekt s načtenými výsledky.

Pro přehledné zobrazení výsledků je třeba je například seřadit, očistit od celých cest k funkcím, atd.
- Odstraňte cesty k funkcím: `.strip_dirs()`
- Zkusme si to seřadit podle celkového času běhu: `.sort_stats("cumulative")`
- Dále můžeme vypsat pouze prvních 10 nejvíce časově náročných funkcí: `.print_stats(10)`.

In [4]:
import pstats
p = pstats.Stats("vysledky_cProfile_code_setup.prof")
p.strip_dirs().sort_stats("cumulative").print_stats(10)

Tue Apr 30 09:32:34 2024    vysledky_cProfile_code_setup.prof

         73985 function calls (71915 primitive calls) in 1.339 seconds

   Ordered by: cumulative time
   List reduced from 722 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     94/1    0.000    0.000    1.339    1.339 {built-in method builtins.exec}
        1    0.002    0.002    1.339    1.339 <string>:1(<module>)
        1    0.000    0.000    1.337    1.337 2558653823.py:15(code_setup)
        1    0.000    0.000    1.101    1.101 2558653823.py:8(heavy_loop)
       20    1.073    0.054    1.101    0.055 2558653823.py:1(heavy_calc)
       12    0.001    0.000    0.271    0.023 __init__.py:1(<module>)
        1    0.000    0.000    0.122    0.122 2558653823.py:21(<listcomp>)
       20    0.122    0.006    0.122    0.006 {method 'rand' of 'numpy.random.mtrand.RandomState' objects}
    111/1    0.001    0.000    0.113    0.113 <frozen importlib._bootstrap>:1022(_find

**Všechno (od profilování až po výpisu výsledku přes pstats) lze v Jupyteru udělat pěkně pomocí magic příkazu:**


In [5]:
%prun -s cumulative -l 10 code_setup()

Will generate 20 random arrays
Will calculate now
Finished calculation
 

         193 function calls in 1.261 seconds

   Ordered by: cumulative time
   List reduced from 28 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.261    1.261 {built-in method builtins.exec}
        1    0.003    0.003    1.261    1.261 <string>:1(<module>)
        1    0.000    0.000    1.258    1.258 2558653823.py:15(code_setup)
        1    0.000    0.000    1.124    1.124 2558653823.py:8(heavy_loop)
       20    1.088    0.054    1.124    0.056 2558653823.py:1(heavy_calc)
        1    0.000    0.000    0.133    0.133 2558653823.py:21(<listcomp>)
       20    0.133    0.007    0.133    0.007 {method 'rand' of 'numpy.random.mtrand.RandomState' objects}
       20    0.036    0.002    0.036    0.002 {method 'copy' of 'numpy.ndarray' objects}
        3    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        6    0.000    0.000    0.000    0.000 iostream.py:500(write)

## Řádkové profilování: Line_profiler
V minulém příkladu jsme viděli, že základní profiler časuje pouze celé funkce. Co když chceme zjistit, které řádky kódu jsou nejvíce časově náročné?

Toto mnoho profilerů umožňuje, například `line_profiler`. (Pokud nemáte `pip install line_profiler`)

Line profiler se dá spouštět z příkazového řádku pro celý file, ale to kdyžtak nechám zájemcům.

My si zde ukážeme jeho použití v Jupyteru pomocí magic příkazu.

In [6]:
# je třebe nejprve načíst příslušný modul
%load_ext line_profiler

Spouští se příkazem `%lprun -f funkce_k_profilovani funkce_k_profilovani(parametry)`
- parametr `-f` určuje funkci, kterou chceme profilovat (může jich být více)
- parametr `-u` určuje jednotku času ve výstupu

In [7]:
%lprun -u 1e-6 -f code_setup -f heavy_loop -f heavy_calc code_setup()

Will generate 20 random arrays
Will calculate now
Finished calculation


Timer unit: 1e-06 s

Total time: 1.05992 s

Could not find file /tmp/ipykernel_19517/2558653823.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           
     2        20      18308.0    915.4      1.7  
     3       220        737.0      3.4      0.1  
     4       200    1040861.0   5204.3     98.2  
     5        20         12.0      0.6      0.0  

Total time: 1.06071 s

Could not find file /tmp/ipykernel_19517/2558653823.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           
     9         1          1.0      1.0      0.0  
    10        21         37.0      1.8      0.0  
    11    

## Moderní, rychlý all-in-one profiler: Scalene
Scalene oproti předchozím umožňuje profilovat jak řádky tak funkce a navíc přidává i profilování paměti. (A GPU pokud používáme)

Vyžaduje instalaci pomocí `pip install scalene`

Spouští se příkazem `scalene script.py`

In [8]:
%%writefile code_profile_test.py
def heavy_calc(X):
    Y = X.copy()
    for i in range(10):
        Y = Y**i
    return Y

def heavy_loop(inputs):
    res = []
    for X in inputs:
        res.append(heavy_calc(X))
    return res

def code_setup():
    from numpy.random import rand
    N = 20
    M = 1000
    print("Will generate {} random arrays".format(N))
    inputs = [rand(M, M) for n in range(N)]
    print("Will calculate now")
    result = heavy_loop(inputs)
    print("Finished calculation")

code_setup()

Writing code_profile_test.py


In [9]:
!!scalene code_profile_test.py

['/bin/bash: line 1: scalene: command not found']

### Scalene v Jupyteru
Scalene se dá použít i v Jupyteru, pomocí modulu `scalene` a magic příkazu `%%scalene`.

In [10]:
%load_ext scalene

The scalene module is not an IPython extension.


In [11]:
%%scalene
def heavy_calc(X):
    Y = X.copy()
    for i in range(10):
        Y = Y**i
    return Y

def heavy_loop(inputs):
    res = []
    for X in inputs:
        res.append(heavy_calc(X))
    return res

def code_setup():
    from numpy.random import rand
    N = 20
    M = 1000
    print("Will generate {} random arrays".format(N))
    inputs = [rand(M, M) for n in range(N)]
    print("Will calculate now")
    result = heavy_loop(inputs)
    print("Finished calculation")

code_setup()

UsageError: Cell magic `%%scalene` not found.
